### Exercício 2 - Pedro Gonçalves (A82313) & Roberto Cachada (A81012)

Para este exercício era pedido a criação de dois agentes, ***Emitter*** e ***Receiver***, de modo a estabelecer uma comunicação privada entre ambos, utilizando para isso a *package* ***Cryptography***. Para tal, foram criadas diferentes funções de modo a cumprir esse objetivo. Essas funções serão descritas em baixo.

##### Alínea a)
* Para esta primeira alínea foram criadas as funções ***encrypt*** e ***decrypt***. Tal como os nomes sugerem, a primeira função serve para encriptar/autenticar o texto limpo utilizando a cifra *AES (Advanced Encryption Standard)*, em modo *GCM (Galois Counter Mode)*, e ao mesmo tempo autenticar metadados extra. A segunda função decifra o resultado produzido pela função *encrypt* e verifica a autenticidade dos metadados.

In [1]:
import os
import getpass
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

def encrypt(chave, textolimpo, metadados):
    iv = os.urandom(12) #gera um IV de 12 bytes

    #Criação de um objeto AES-GCM através da chave e do IV 
    encryptor = Cipher(
        algorithms.AES(chave), #Cifra AES
        modes.GCM(iv), #Modo GCM
        backend=default_backend()
    ).encryptor()

    #Autentica os metadados
    encryptor.authenticate_additional_data(metadados)

    #Encripta o texto limpo
    ciphertext = encryptor.update(textolimpo) + encryptor.finalize()

    return (iv, ciphertext, encryptor.tag)

def decrypt(chave, metadados, iv, ciphertext, tag):
    #Criação de um objeto AES-GCM através da chave, do IV e da tag dos metadados
    decryptor = Cipher(
        algorithms.AES(chave),
        modes.GCM(iv, tag),
        backend=default_backend()
    ).decryptor()

    #Verifica a autenticidade dos metadados
    decryptor.authenticate_additional_data(metadados)

    #Retorna o texto limpo
    return decryptor.update(ciphertext) + decryptor.finalize()

##### Alínea b)
* Nesta alínea foi criada a função ***kdf*** que dado uma palavra-passe e um *salt* de *16 bytes*, gera uma chave de *32 bytes* segura, para ser utilizada nos processos de encriptação/decifragem, a partir de uma palavra-passe facilmente memorizável pelos utilizadores. 

In [2]:
def kdf(password, salt): #Recebendo um palavra-passe e um salt de 16 bytes gera uma chave para utilizar nos processos de encriptação/decifragem
    kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(), #É ulilizado o SHA256
    length=32, #Comprimento da chave gerada
    salt=salt,
    iterations=100000,
    backend=default_backend()
    )
    key = kdf.derive(password.encode('utf-8'))
    return key

##### Alínea c)
* Para garantir a autenticação prévia da chave gerada pela função em cima, foi criada a função ***mac***, que utiliza uma função de *hash* criptográfica juntamente com uma chave para cumprir tal objetivo. Esta função serve também para verificar se a autenticidade da chave durante o processo de decifragem.

In [3]:
def mac(chave,texto, tag=None):
    h = hmac.HMAC(chave,hashes.SHA256(),default_backend())
    h.update(texto)
    if tag == None: #Se não existir tag, autentica o texto
        return h.finalize()
    h.verify(tag) #Caso contrário verifica a autenticidade da tag

##### Comunicação Privada
* Para os agentes comunicarem entre si, é criado uma ligação através de um *pipe*, que permite ao ***Emitter*** enviar uma *stream* de informação encriptada ao ***Receiver***, fechando ambos a conexão após o envio/receção da *stream* de informação.

In [4]:
from multiprocessing import Process, Pipe

class PipeCom(object):
    def __init__(self,left,right): #Recebe os agentes
        left_end, right_end = Pipe() #São criadas as "pontas" do Pipe
        self.left  = lambda : left(left_end) #Define quais as "pontas" a que cada agente tem acesso
        self.right = lambda : right(right_end)
        
    def execute(self): #Executa os agentes
        self.left()
        self.right()

In [5]:
salt = os.urandom(16) #Salt partilhado
metadados = b"metadados vfvfvfvfv"

def Receiver(conn):
    password = getpass.getpass('Password Receiver:')
    keyR = kdf(password,salt) #Cria chave para do Receiver

    try: #Separar os diferentes componentes da stream(dicionário) de dados recebida
        data = conn.recv() #Recebe dados
        ciphertext = data['cipher']
        tagM = data['tagM']
        tagK = data['tagK']
        iv = data['iv']
    
        mac(keyR,keyR,tagK) #Verifica o MAC da chave
        print("MAC chave verificado")
        print(decrypt( #Decifra o ciphertext e verifica a autenticidade dos metadados
            keyR,
            metadados,
            iv,
            ciphertext,
            tagM
        ))
        print("OK")
    except:
        print("Erro no Receiver")
    conn.close() #Fecha a sua "ponta" do Pipe

def Emitter(conn):
    password = getpass.getpass('Password Emitter')
    keyE = kdf(password,salt) #Cria chave para do Emitter
    texto = input('Mensagem a Cifrar:')
    try:
        iv, ciphertext, tagM = encrypt( #Cifra texto limpo + MAC metadados
            keyE,
            texto.encode('utf-8'),
            metadados
        )
        tagK = mac(keyE,keyE) #MAC chave
        data = {'cipher' : ciphertext , 'tagM' : tagM , 'tagK' : tagK, 'iv' : iv} #Organiza os elementos numa Stream(dicionário)
        conn.send(data) #Envia dados
        print("Dados Enviados")
    except:
        print("Erro no Emitter")
    conn.close() #Fecha a sua "ponta" do Pipe

In [ ]:
PipeCom(Emitter,Receiver).execute()